In [16]:
from pymot import MOTEvaluation
from os import path, listdir
from collections import defaultdict

In [17]:
result_path = "../dataset/results/mot17_val_motdt"
data_path = "../dataset/train"

In [18]:
def mot(testname, groundtruth, hypotheses, MOTA):
    evaluator = MOTEvaluation(groundtruth, hypotheses, 0.2)
    evaluator.evaluate()
    MOTA[testname] = (evaluator.getMOTA(), evaluator.getMOTP())
    print("MOTA: ", evaluator.getMOTA())
    print("MOTP: ", evaluator.getMOTP())
    print(evaluator.getRelativeStatistics())
    print(evaluator.getAbsoluteStatistics())

In [19]:
def load_result(data_file, result_file):
    len_seq = 0
    with open(path.join(data_file, "seqinfo.ini")) as f:
        len_seq = int(f.readlines()[4].strip().split("=")[1])
    
    gt = [[] for i in range(len_seq + 1)]
    result = [[] for i in range(len_seq + 1)]
    with open(path.join(data_file, 'gt', 'gt.txt')) as f:
        for seq, *data in [line.strip().split(',') for line in f.readlines()]:
            gt[int(seq)].append(data)
    
    with open(result_file) as f:
        for seq, *data in [line.strip().split(',') for line in f.readlines()]:
            result[int(seq)].append(data)

    return gt, result

In [20]:
def result_to_json(gt, result):
    hypotheses = {}
    groundtruth = {}
    num = 0
    hypotheses['frames'] = []
    hypotheses['class'] = "video"
    hypotheses['filename'] = "../MOT/labs/mot17.idx"
    groundtruth['frames'] = []
    groundtruth['class'] = "video"
    groundtruth['filename'] = "../MOT/labs/mot17.idx"
    for g in gt:
        groundtruth["frames"].append({"timestamp": num})
        groundtruth["frames"][num]["num"] = num
        groundtruth["frames"][num]["class"] = "frame"
        groundtruth["frames"][num]["annotations"] = []
        for obj in g:
            if num == 1:
                continue
            idx, x, y, w, h = obj[0], float(obj[1]), float(obj[2]), float(obj[3]), float(obj[4])
            if w < 0 or h < 0 :
                #print("wrong value in gt")
                #print(obj)
                continue
            groundtruth["frames"][num]["annotations"].append({"dco": False, "height": h, "width": w, "id": idx, "y": y, "x":x})
        num += 1
        
    num = 0
    for r in result:
        hypotheses["frames"].append({"timestamp": num})
        hypotheses["frames"][num]["num"] = num
        hypotheses["frames"][num]["class"] = "frame"
        hypotheses["frames"][num]["hypotheses"] = []
        for obj in r:
            idx, x, y, w, h = obj[0], float(obj[1]), float(obj[2]), float(obj[3]), float(obj[4])
            if w < 0 or h < 0 :
                #print("wrong value in det")
                #print(obj)
                continue
            hypotheses["frames"][num]["hypotheses"].append({"height": h, "width": w, "id": idx, "y": y, "x":x})
        num+=1
    return groundtruth, hypotheses

In [21]:
MOTA = {}

In [22]:
for f in listdir(data_path):
    gt, result = load_result(path.join(data_path,f), path.join(result_path ,f+".txt"))
    groundtruth, hypotheses = result_to_json(gt, result)
    mot(f, groundtruth, hypotheses, MOTA)
    print(MOTA[f])

0 37 0 29964
0 37 0 29964
MOTA:  0.9987651848885329
MOTP:  0.6870555177877732
0 37 0 29964
{'track recall': 0.6144578313253012, 'mismatch rate': 0.0, 'MOTP': 0.6870555177877732, 'non-recoverable mismatch rate': 0.0, 'recoverable mismatch rate': 0.0, 'MOTA': 0.9987651848885329, 'track precision': 0.978021978021978, 'miss rate': 0.0, 'false positive rate': 0.0012348151114670938}
{'false positives': 37, 'hypothesis tracks': 91, 'non-recoverable mismatches': 0, 'covering hypothesis tracks': 89, 'misses': 0, 'ground truths': 29964, 'mismatches': 0, 'covered ground truth tracks': 51, 'ground truth tracks': 83, 'lonely ground truth tracks': 32, 'recoverable mismatches': 0, 'total overlap': 4488.533697707522, 'correspondences': 6533, 'lonely hypothesis tracks': 2}
(0.9987651848885329, 0.6870555177877732)
0 13 2 29964
0 13 2 29964
MOTA:  0.999499399279135
MOTP:  0.8152262874925009
0 13 2 29964
{'track recall': 0.6144578313253012, 'mismatch rate': 6.674676278200507e-05, 'MOTP': 0.815226287492500

KeyboardInterrupt: 

In [15]:
MOTA

{'MOT17-02-DPM': (0.5757242023761848, 0.5722082007551219)}

In [24]:
sum_mota = 0
sum_motp = 0
for m in MOTA.keys():
    sum_mota += MOTA[m][0]
    sum_motp += MOTA[m][1]
print(sum_mota / len(MOTA))
print(sum_motp / len(MOTA))

0.3562686693739573
0.7742206158905386
